In [1]:
import os
import glob
import math
import pickle

import numpy as np
import pandas as pd
import geopandas as gpd
import torch

import processing_io as pio
from torch_geometric.transforms import LineGraph

from torch_geometric.data import Data, Batch

highway_mapping = {
    'trunk': 0, 'trunk_link': 0, 'motorway_link': 0,
    'primary': 1, 'primary_link': 1,
    'secondary': 2, 'secondary_link': 2,
    'tertiary': 3, 'tertiary_link': 3,
    'residential': 4, 'living_street': 5,
    'pedestrian': 6, 'service': 7,
    'construction': 8, 'unclassified': 9,
    'np.nan': -1
}

# Parameters to define
is_for_1pm = True
result_df_name = 'dataset_1pm_0-5000'

result_path = '../../../../data/datasets_simulation_outputs/' + result_df_name + '.pt'
string_is_for_1pm = "pop_1pm" if is_for_1pm else "pop_1pct"

base_dir = '../../../../data/' + string_is_for_1pm + '_simulations/' + string_is_for_1pm + '_with_policies/'
subdirs_pattern = os.path.join(base_dir, 'output_networks_*')
subdirs = list(set(glob.glob(subdirs_pattern)))
subdirs.sort()
if is_for_1pm:
    gdf_basecase_mean = gpd.read_file("results/gdf_basecase_mean_pop_1pm.geojson")
else:
    gdf_basecase_mean = gpd.read_file('results/gdf_' + string_is_for_1pm + '_basecase_mean.geojson')

## Process results

Process the outputs of the simulations for further usage by GNN.

In [2]:
# Read all network data into a dictionary of GeoDataFrames
result_dic = {}
counter =0

# Loop through each subdirectory
for subdir in subdirs:
    if "filtered" in subdir:
        continue
    # if "output_networks_3200" not in subdir:
    #     continue
    print(f'Accessing folder: {subdir}')
    print(len(os.listdir(subdir)))
    networks = [network for network in os.listdir(subdir) if not network.endswith(".DS_Store")]
    print(len(networks))

    for network in networks:
        file_path = os.path.join(subdir, network)
        policy_key = pio.create_policy_key_1pm(network)
        gdf = pio.read_output_links(file_path)
        
        if (gdf is not None):
            gdf_extended = pio.extend_geodataframe(gdf_base=gdf_basecase_mean, gdf_to_extend=gdf, column_to_extend='highway', new_column_name='highway')
            gdf_extended = pio.extend_geodataframe(gdf_base=gdf_basecase_mean, gdf_to_extend=gdf_extended, column_to_extend='vol_car', new_column_name='vol_car_base_case')
            result_dic[policy_key] = gdf_extended
base_network_no_policies = gdf_basecase_mean
        
result_dic["base_network_no_policies"] = base_network_no_policies

Accessing folder: ../../../../data/pop_1pm_simulations/pop_1pm_with_policies/output_networks_100
101
100
Accessing folder: ../../../../data/pop_1pm_simulations/pop_1pm_with_policies/output_networks_1000
100
100
Accessing folder: ../../../../data/pop_1pm_simulations/pop_1pm_with_policies/output_networks_1100
100
100
Accessing folder: ../../../../data/pop_1pm_simulations/pop_1pm_with_policies/output_networks_1200
100
100
Accessing folder: ../../../../data/pop_1pm_simulations/pop_1pm_with_policies/output_networks_1300
100
100
Accessing folder: ../../../../data/pop_1pm_simulations/pop_1pm_with_policies/output_networks_1400
100
100
Accessing folder: ../../../../data/pop_1pm_simulations/pop_1pm_with_policies/output_networks_1500
100
100
Accessing folder: ../../../../data/pop_1pm_simulations/pop_1pm_with_policies/output_networks_1600
100
100
Accessing folder: ../../../../data/pop_1pm_simulations/pop_1pm_with_policies/output_networks_1700
100
100
Accessing folder: ../../../../data/pop_1pm_simu

## Analyze results and plot

In [3]:
# pio.analyze_geodataframes(result_dic=result_dic, consider_only_highway_edges=True)

In [4]:
# pio.analyze_geodataframes(result_dic=result_dic, consider_only_highway_edges=False)

In [5]:
def process_result_dic(result_dic):
    datalist = []
    linegraph_transformation = LineGraph()
    base_network_no_policies = result_dic.get("base_network_no_policies")
    vol_base_case = base_network_no_policies['vol_car'].values
    capacity_base_case = base_network_no_policies['capacity'].values
    
    # Initialize base edge positions
    gdf_base = gpd.GeoDataFrame(base_network_no_policies, geometry='geometry')
    gdf_base.crs = "EPSG:2154"  # Assuming the original CRS is EPSG:2154
    gdf_base.to_crs("EPSG:4326", inplace=True)
    edge_positions_base = np.array([((geom.coords[0][0] + geom.coords[-1][0]) / 2, 
                                     (geom.coords[0][1] + geom.coords[-1][1]) / 2) 
                                    for geom in gdf_base.geometry])
    
    nodes = pd.concat([gdf_base['from_node'], gdf_base['to_node']]).unique()
    node_to_idx = {node: idx for idx, node in enumerate(nodes)}
    gdf_base['from_idx'] = gdf_base['from_node'].map(node_to_idx)
    gdf_base['to_idx'] = gdf_base['to_node'].map(node_to_idx)
    edges_base = gdf_base[['from_idx', 'to_idx']].values
    edge_positions_tensor = torch.tensor(edge_positions_base, dtype=torch.float)
    edge_index = torch.tensor(edges_base, dtype=torch.long).t().contiguous()
    x = torch.zeros((len(nodes), 1), dtype=torch.float)
    data = Data(edge_index=edge_index, x=x, pos=edge_positions_tensor)
    linegraph_data = linegraph_transformation(data)

    for key, df in result_dic.items():        
        if isinstance(df, pd.DataFrame) and key != "base_network_no_policies":
            gdf = gpd.GeoDataFrame(df, geometry='geometry')
            gdf.crs = "EPSG:2154"  # Assuming the original CRS is EPSG:2154
            gdf.to_crs("EPSG:4326", inplace=True)

            capacities_new = gdf['capacity'].values
            capacity_reduction = capacities_new - capacity_base_case
            # capacity_percentage_reduction = (capacities - capacity_base_case)/capacity_base_case
            
            highway = gdf['highway'].apply(lambda x: highway_mapping.get(x, -1)).values

            edge_car_volume_difference = gdf['vol_car'].values - vol_base_case
            
            target_values = torch.tensor(edge_car_volume_difference, dtype=torch.float).unsqueeze(1)

            # Transform to line graph
            linegraph_x = torch.tensor(np.column_stack((vol_base_case, capacity_base_case, capacities_new, capacity_reduction, highway)), dtype=torch.float)
            linegraph_data.x = linegraph_x
            
            # Target tensor for car volumes
            linegraph_data.y = target_values
            if linegraph_data.validate(raise_on_error=True):
                datalist.append(linegraph_data)
            else:
                print("Invalid line graph data")
                
    # Convert dataset to a list of dictionaries
    data_dict_list = [{'x': lg_data.x, 'edge_index': lg_data.edge_index, 'pos': lg_data.pos, 'y': lg_data.y} for lg_data in datalist]
    return data_dict_list

## Save for further processing with GNN

In [6]:
data_processed = process_result_dic(result_dic)
# data_processed_single_districts = pio.process_result_dic(result_dic_single_districts)
# torch.save(data_processed_single_districts, result_path + '_single_districts.pt')

In [7]:
torch.save(data_processed, result_path)